In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch" # Comment out for tensorflow backend

from molexpress import layers
from molexpress.datasets import featurizers
from molexpress.datasets import encoders
from molexpress.ops.chem_ops import get_molecule

import torch

## 1. Featurizers

In [2]:
mol = get_molecule('C(C(=O)O)N')

print(featurizers.AtomType(vocab={'O'}, oov=False)(mol.GetAtoms()[0]))
print(featurizers.AtomType(vocab={'O'}, oov=True)(mol.GetAtoms()[0]))
print(featurizers.AtomType(vocab={'C', 'O'}, oov=False)(mol.GetAtoms()[0]))
print(featurizers.AtomType(vocab={'C', 'O', 'N'}, oov=False)(mol.GetAtoms()[0]))
print(featurizers.AtomType(vocab={'C', 'O', 'N'}, oov=True)(mol.GetAtoms()[0]))

[0.]
[0. 1.]
[1. 0.]
[1. 0. 0.]
[1. 0. 0. 0.]


## 2. Encoder

In [3]:
atom_featurizers = [
    featurizers.AtomType({'C', 'O', 'N'}),
    featurizers.Hybridization(),
]

bond_featurizers = [
    featurizers.BondType()
]

peptide_graph_encoder = encoders.PeptideGraphEncoder(
    atom_featurizers=atom_featurizers,
    bond_featurizers=bond_featurizers,
    self_loops=True # adds one dim to edge state
)

mol2 = get_molecule('CC(C(=O)O)N')

peptide_graph_encoder([mol, mol2])

{'node_state': array([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32),
 'edge_state': array([[0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0.

## 3. Dataset

In [4]:
x_dummy = [
    ['CC(C)C(C(=O)O)N', 'C(C(=O)O)N'],
    ['C(C(=O)O)N', 'CC(C(=O)O)N', 'C(C(=O)O)N'],
    ['CC(C(=O)O)N']
]
y_dummy = [1., 2., 3.]


class TinyDataset(torch.utils.data.Dataset):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        x = peptide_graph_encoder(x)
        return x, [y]

torch_dataset = TinyDataset(x_dummy, y_dummy)

dataset = torch.utils.data.DataLoader(
    torch_dataset, batch_size=2, collate_fn=peptide_graph_encoder.collate_fn)

for x, y in dataset:
    print(f'x = {x}\ny = {y}', end='\n' + '---' * 30 + '\n')

x = {'node_state': array([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
 

## 4. Model

In [5]:
class TinyGCNModel(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.gcn1 = layers.GINConv(32)
        self.gcn2 = layers.GINConv(32)
        self.readout = layers.ResidueReadout()
        self.lstm = torch.nn.LSTM(32, 32, 1, batch_first=True)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x):
        x = self.gcn1(x)
        x = self.gcn2(x)
        x = self.readout(x)
        x, (_, _) = self.lstm(x)
        x = self.linear(x[:, -1, :])
        return x

model = TinyGCNModel()#.to('cuda')

## 5. Fit

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = torch.nn.MSELoss()

for _ in range(30):
    loss_sum = 0.
    for x, y in dataset:
        optimizer.zero_grad()
        outputs = model(x)
        y = torch.tensor(y, dtype=torch.float32)#.to('cuda')
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss

    print(loss_sum)

tensor(12.4940, grad_fn=<AddBackward0>)
tensor(11.6400, grad_fn=<AddBackward0>)
tensor(10.2537, grad_fn=<AddBackward0>)
tensor(8.7948, grad_fn=<AddBackward0>)
tensor(7.4848, grad_fn=<AddBackward0>)
tensor(6.2963, grad_fn=<AddBackward0>)
tensor(5.2063, grad_fn=<AddBackward0>)
tensor(4.2183, grad_fn=<AddBackward0>)
tensor(3.3452, grad_fn=<AddBackward0>)
tensor(2.5991, grad_fn=<AddBackward0>)
tensor(1.9870, grad_fn=<AddBackward0>)
tensor(1.5088, grad_fn=<AddBackward0>)
tensor(1.1574, grad_fn=<AddBackward0>)
tensor(0.9188, grad_fn=<AddBackward0>)
tensor(0.7738, grad_fn=<AddBackward0>)
tensor(0.7003, grad_fn=<AddBackward0>)
tensor(0.6754, grad_fn=<AddBackward0>)
tensor(0.6783, grad_fn=<AddBackward0>)
tensor(0.6919, grad_fn=<AddBackward0>)
tensor(0.7038, grad_fn=<AddBackward0>)
tensor(0.7066, grad_fn=<AddBackward0>)
tensor(0.6975, grad_fn=<AddBackward0>)
tensor(0.6766, grad_fn=<AddBackward0>)
tensor(0.6462, grad_fn=<AddBackward0>)
tensor(0.6096, grad_fn=<AddBackward0>)
tensor(0.5702, grad_fn